In [3]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal
import sympy as sym
import numpy as np
import scipy as sp

from sympy import sin, cos

sns.set()
plt.rcParams["figure.figsize"] = (8,8)

# Part A


Consider a FIR (finite impulse response) discrete time filter such that

$$
    x_n \sum\limits_{m=0}^{N_h-1} h_m g_{n-m}
$$

$x_n$ is the filter output, $h_m$ are the FIR filter coefficients and, $g_n$ is the input excitation sequence which is assumed to be known. It is convenient if it is a white noise sequence of normal random samples of unit variance and zero mean. A common problem is that $x_n$ is corrupted by additive noise such that the actual observation is


$$
    x_n \sum\limits_{m=0}^{N_h-1} h_m g_{n-m} + w_n
$$

where

$$
    w \sim \mathcal{N}(0, \sigma^2)
$$

Write a routine that estimates the values of $h_m$ given an input excitation sequence of $g_n$ that is assumed to be known and with
$w_n$ as iid (independent and identically distributed) zero mean Gaussian with
$\sigma^2 = 0.1$. Try out your routine with the filter coefficients of: `h = [1,2,-1,4,1]`;

In [61]:
N = 1000
h = np.array([1, 2, -1, 4, 1])
g = np.random.randn(N)
sigma = 0.1

x_conv = sp.signal.convolve(g, h)
H = np.zeros((N + len(h) - 1, len(h)))

In [62]:
for n in range(N + len(h)):
    for m in range(len(h)):
        g_idx = n - m
        if 0 <= g_idx < N:
            H[n,m] = g[g_idx]

In [6]:
x_H = H @ h.T

In [7]:
np.sum(x_H - x_conv)

-1.1435297153639112e-14

In [8]:
x = H @ h.T
x += np.random.randn(*x.shape) * sigma

### The least squares solution

The least squares solution is:
$$
    \hat h = \hat \theta = (H^T H)^{-1} H^T x
$$

In [9]:
h_hat = (np.linalg.inv(H.T @ H)) @ H.T @ x

In [10]:
# All values should be close
h, h_hat

(array([ 1,  2, -1,  4,  1]),
 array([ 1.01319546,  1.9689184 , -0.99873338,  4.00360459,  1.01374515]))

In [166]:
# Create function that will be used to rerun experiment several times

def gen_H(N=1000, h=[1, 2, -1, 4, 1], g=None):
    if g is None:
        g = np.random.randn(N)

    # Generate H
    H = np.zeros((N + len(h) - 1, len(h)))
    for n in range(N + len(h)):
        for m in range(len(h)):
            g_idx = n - m
            if 0 <= g_idx < N:
                H[n,m] = g[g_idx]
    return H


def gen_x(H, h=[1, 2, -1, 4, 1], Q=None, sigma=None):
    # Create a x ~ N(sum(h_m * g_{n-m}), Q)
    h = np.asarray(h)
    x = H @ h.T  # Mean of samples

    if sigma is not None:
        x += np.random.randn(*x.shape) * sigma
    elif Q is not None:
        x = np.random.multivariate_normal(x, Q)
    else:
        raise ArgumentError('One of [Q, sigma] must be given')

    return x


def h_hat_fn(H, x, Q):
    return (np.linalg.inv(H.T @ H)) @ H.T @ x
        

def run_experiment(h=[1, 2, -1, 4, 1], h_hat_fn=h_hat_fn, g=None, k=1000, N=1000, Q=None, sigma=None):
    '''Using a certain h (and optionally g) run the experiment k times.'''
    h_hats = np.zeros((k, len(h)))
    for i in range(k):
        H = gen_H(N=N)
        x = gen_x(H=H, h=h, Q=Q, sigma=sigma)
        h_hats[i,:] = h_hat_fn(H=H, x=x, Q=Q)
    return h_hats
    
        
def calc_avg_deviation(h, h_hats):
    return np.mean(np.abs(h_hats - h), axis=0)
    

In [74]:
# Calculate the avg deviation from 100 runs
h_hats = run_experiment(h=[1, 2, -1, 4, 1], h_hat_fn=h_hat_fn, g=None, k=1000, N=1000, Q=None, sigma=0.1)
avg_dev = calc_avg_deviation(h, h_hats)

In [75]:
avg_dev

array([0.00260733, 0.0025125 , 0.00253874, 0.00266757, 0.00246098])

# Part B

What is the expected deviation of the FIR coefficients in part A based on Fisher Information assuming that N=1000 data samples are used. Is there agreement?

In [189]:
def run_experiment_B(h=[1, 2, -1, 4, 1], h_hat_fn=h_hat_fn, g=None, k=1000, N=1000, Q=None, sigma=None):
    '''Using a certain h (and optionally g) run the experiment k times.'''
    exp_devs = np.zeros((k, len(h)))
    sqrt_crlb_devs = np.zeros((k, len(h)))
    for i in range(k):
        H = gen_H(N=N)
        x = gen_x(H=H, h=h, Q=Q, sigma=sigma)
        h_hat = h_hat_fn(H=H, x=x, Q=Q)
        exp_devs[i,:] = np.abs(h - h_hat)
        
        if sigma is not None:
            J = H.T @ H / (sigma**2) # Equivalent to the H^T Q^{-1} H since Q is diagonal
        elif Q is not None:
            J = H.T @ np.linalg.inv(Q) @ H
        else:
            raise ArgumentError('One of [Q, sigma] must be given')

        J_inv = np.linalg.inv(J)
        sqrt_crlb_devs[i,:] = np.sqrt(np.diag(J_inv))

    return exp_devs, sqrt_crlb_devs


In [191]:
exp_devs, sqrt_crlb_devs = run_experiment_B(h=[1, 2, -1, 4, 1], h_hat_fn=h_hat_fn, k=1000, N=1000, sigma=0.1)

print(f'nexperimental_dev: {np.mean(exp_devs, axis=0)}')
print(f'expected_dev:     {np.mean(crlb_devs, axis=0)}')
# print(f'experimental_dev: {np.mean(exp_devs**2, axis=0)}')
# print(f'expected_dev:     {np.mean(crlb_devs**2, axis=0)}')

nexperimental_dev: [0.00259953 0.00253518 0.0026013  0.00258036 0.00245859]
expected_dev:     [0.0031675  0.0031675  0.00316743 0.0031675  0.0031675 ]


# Part C

In [177]:
seq_len = H.shape[0]
Q_c = np.zeros((seq_len, seq_len))

for n in range(seq_len):
    for m in range(seq_len):
        Q_c[n,m] = sigma**2 * np.exp(-np.abs(m-n))

[0.01       0.00367879 0.00135335 ... 0.         0.         0.        ]


In [183]:
print(Q_c[:5,:5])

[[0.01       0.00367879 0.00135335 0.00049787 0.00018316]
 [0.00367879 0.01       0.00367879 0.00135335 0.00049787]
 [0.00135335 0.00367879 0.01       0.00367879 0.00135335]
 [0.00049787 0.00135335 0.00367879 0.01       0.00367879]
 [0.00018316 0.00049787 0.00135335 0.00367879 0.01      ]]


In [160]:
Q_c_inv = np.linalg.inv(Q_c)

In [161]:
# Generate new data with the new covariance
x = H @ h.T
x = np.random.multivariate_normal(x, Q_c)

In [162]:
h_hat_gmf = (np.linalg.inv(H.T @ Q_c_inv @ H)) @ H.T @ Q_c_inv @ x

In [163]:
h_hat_gmf

array([ 1.00323162,  2.00049214, -0.99300842,  4.00166193,  1.00095349])

In [168]:
def h_hat_gmf_fn(H, x, Q):
    print(Q.shape)
    Q_inv = np.linalg.inv(Q)
    h_hat = (np.linalg.inv(H.T @ Q_inv @ H)) @ H.T @ Q_inv @ x
    print(h_hat)
    return h_hat
    
# Calculate the avg deviation from 100 runs
# print(Q_c)
h_hats = run_experiment(h=[1, 2, -1, 4, 1], h_hat_fn=h_hat_gmf_fn, k=100, N=1000, Q=Q_c)
avg_dev = calc_avg_deviation(h, h_hats)

[[0.01       0.00367879 0.00135335 ... 0.         0.         0.        ]
 [0.00367879 0.01       0.00367879 ... 0.         0.         0.        ]
 [0.00135335 0.00367879 0.01       ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.01       0.00367879 0.00135335]
 [0.         0.         0.         ... 0.00367879 0.01       0.00367879]
 [0.         0.         0.         ... 0.00135335 0.00367879 0.01      ]]
(1004, 1004)
[ 1.00245978  1.99940632 -0.99556113  3.99856616  1.0012277 ]
(1004, 1004)
[ 0.99976527  2.00029551 -1.00045237  4.00175026  1.00118255]
(1004, 1004)
[ 1.00494485  1.99981835 -0.99905237  4.00111097  0.99855061]
(1004, 1004)
[ 0.99867222  1.99313392 -1.00063559  3.99545211  0.99739584]
(1004, 1004)
[ 1.0037813   1.99932894 -0.99457002  4.00263327  1.00114628]
(1004, 1004)
[ 0.99972566  1.99852105 -0.99901691  4.00399102  1.00268306]
(1004, 1004)
[ 0.99510709  1.99819931 -1.00119782  3.99296562  0.99876482]
(1004, 1004)
[ 1.00072314  2.00

In [174]:
J = H.T @ Q_c_inv @ H
J_inv = np.linalg.inv(J)
crlb_dev = np.sqrt(np.diag(J_inv))
                
avg_dev, crlb_dev

(array([0.00245007, 0.00242708, 0.00243574, 0.00251556, 0.00234289]),
 array([0.00293986, 0.00319976, 0.00321227, 0.00319976, 0.00293986]))

# Part D

In [19]:
seq_len = H.shape[0]
Q_d = np.zeros((seq_len, seq_len))

for n in range(seq_len):
    for m in range(seq_len):
        Q_d[n,m] = sigma**2 * np.exp(-0.1 * np.abs(m-n))

In [20]:
Q_d_inv = np.linalg.inv(Q_d)

In [21]:
# Generate new data with the new covariance
x = H @ h.T
x = np.random.multivariate_normal(x, Q_d)

In [22]:
h_hat_gmf = (np.linalg.inv(H.T @ Q_d_inv @ H)) @ H.T @ Q_d_inv @ x

In [23]:
h_hat_gmf

array([ 0.99940178,  1.9998907 , -0.99979081,  3.99949386,  0.99940045])